In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn==1.2.2 mlxtend


# This is to disable annoying warning messages from sklearn 1.2.2
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

# Assignment 6
## Ziad Arafat

### Reading in the data

1. We read in the CSV using the pandas library and store it in a dataframe. 
2. We print the data in the first two rows using the `head()` method

In [3]:
"""  
We will use a dataset called Online Retail Data Set from the UCI repository
(https://archive.ics.uci.edu/ml/datasets/Online+Retail). Its csv file (Online Retail.csv) can be
downloaded from Canvas. This is a transactional data set that contains all the transactions
occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online
retail. Note that the InvoiceNo column contains the id of the transaction and the column StockCode
contains the ids of the items.
Based on association rules generation, what are the top 10 association rules (please use
confidence as the rule evaluation metric) that you can find based on this dataset?
For Association Rules Mining, you can consider using frequent_patterns package in MLxtend.
Please check http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/ for how
to generate the association rules using MLxtend.
"""

'  \nWe will use a dataset called Online Retail Data Set from the UCI repository\n(https://archive.ics.uci.edu/ml/datasets/Online+Retail). Its csv file (Online Retail.csv) can be\ndownloaded from Canvas. This is a transactional data set that contains all the transactions\noccurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online\nretail. Note that the InvoiceNo column contains the id of the transaction and the column StockCode\ncontains the ids of the items.\nBased on association rules generation, what are the top 10 association rules (please use\nconfidence as the rule evaluation metric) that you can find based on this dataset?\nFor Association Rules Mining, you can consider using frequent_patterns package in MLxtend.\nPlease check http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/ for how\nto generate the association rules using MLxtend.\n'

In [4]:
df_online_retail = pd.read_csv("Online Retail.csv")
print(df_online_retail.head(n=5))

# columns: InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  


In [5]:
# Step 1: Verify the format of StockCode values
print("Sample StockCode values from the DataFrame:")
print(df_online_retail['StockCode'].head())



Sample StockCode values from the DataFrame:
0    85123A
1     71053
2    84406B
3    84029G
4    84029E
Name: StockCode, dtype: object


### Preprocessing
#### Scaling
1. We use minmax to normalize the data

In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Create the stockcode to description mapping dictionary
# This is to map the StockCode values to the actual description at the end
stockcode_description_mapping = pd.Series(
        df_online_retail.Description.values,
        index=df_online_retail.StockCode
).to_dict()

# Prepare the transactions data
transactions = df_online_retail.groupby('InvoiceNo')['StockCode'].apply(list)

# Transaction Encoder using the actual StockCode values
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_transaction_encoder = pd.DataFrame(te_ary, columns=te.columns_)

# Generate frequent itemsets and association rules
frequent_itemsets = fpgrowth(
        df_transaction_encoder,
        min_support=0.01,
        use_colnames=True
)
rules = association_rules(
        frequent_itemsets,
        metric="confidence",
        min_threshold=0.7
)

#  Map antecedents and consequents to descriptions
rules['antecedents'] = rules['antecedents'].apply(
        lambda x: [stockcode_description_mapping[item] for item in x]
)
rules['consequents'] = rules['consequents'].apply(
        lambda x: [stockcode_description_mapping[item] for item in x]
)

# sort the rules by confidence
rules = rules.sort_values(by='confidence', ascending=False)

# Print the top 10 association rules 
# only the antecedents, consequents and confidence 
# Without truncating the columns
with pd.option_context('display.max_colwidth', None):
    display(rules[['antecedents', 'consequents', 'confidence']].head(n=10))

,antecedents,consequents,confidence
71,"[REGENCY CAKESTAND 3 TIER, PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER ]",[GREEN REGENCY TEACUP AND SAUCER],0.899110
82,[REGENCY TEA PLATE PINK],[REGENCY TEA PLATE GREEN ],0.898089
26,"[SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETROSPOT PAPER NAPKINS ]",[SET/6 RED SPOTTY PAPER PLATES],0.895270
75,"[PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER ]",[GREEN REGENCY TEACUP AND SAUCER],0.894137
69,"[GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY TEACUP AND SAUCER, REGENCY CAKESTAND 3 TIER]",[ROSES REGENCY TEACUP AND SAUCER ],0.875723
22,"[JUMBO STORAGE BAG SUKI, JUMBO BAG PINK POLKADOT, JUMBO SHOPPER VINTAGE RED PAISLEY]",[JUMBO BAG RED RETROSPOT],0.867742
83,[REGENCY TEA PLATE PINK],[REGENCY TEA PLATE ROSES ],0.866242
67,"[REGENCY CAKESTAND 3 TIER, PINK REGENCY TEACUP AND SAUCER]",[GREEN REGENCY TEACUP AND SAUCER],0.860697
10,"[CHARLOTTE BAG PINK POLKADOT, STRAWBERRY CHARLOTTE BAG]",[RED RETROSPOT CHARLOTTE BAG],0.858639
17,"[CHARLOTTE BAG SUKI DESIGN, WOODLAND CHARLOTTE BAG, STRAWBERRY CHARLOTTE BAG]",[RED RETROSPOT CHARLOTTE BAG],0.858553
